# Tarea 3: Piping y Bombas
### Pablo Correa e Ian Gross

## Librerías

In [42]:
import CoolProp.CoolProp as cp
import fluids as fld
import numpy as np
import scipy.constants as cte
import scipy.optimize as opt
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize

from fluids.units import *

In [21]:
g = cte.g*u.m/u.s**2
Patm = (1*u.atm).to(u.Pa)

# Pregunta 1
Considere el sistema de almacenamiento de energía visto en el laboratorio.

A partir de los datos de operación del archivo anexo, verifique si la pérdida de carga observada se puede representar por la ecuación de Ergun.

¿Cuánto es el error con respecto a la correlación? (3pts), ¿Cuanto afecta la temperatura y la variabilidad de las propiedades termofísicas del aire? (3pts).

Asuma que el sistema de almacenamiento es un intercambiador de calor de flujo paralelo, donde el solido es uno de los medio de transferencia de calor. ¿Cuanto sería la efectividad del sistema (intercambiador de calor)? (3pts)

In [113]:
headers = ["Scan Sweep Time (s)", "Temp. Entrada (°C)", "Vel. Aire (m/s)", "Temp. Aire (°C)","Presión Dif. (mbar)", "Temp. Salida (°C)"]
data = pd.read_excel("Ensayo Piloto Lleno T-220°C.xlsx", skiprows=16, usecols="A,C:G", names=headers)

P_0 = 101325
T_0 = 273.15

P = pd.DataFrame(data['Presión Dif. (mbar)'] * 100, columns=["Presión Dif. (mbar)"])
P = P.rename(columns={"Presión Dif. (mbar)": "Presión Dif. (Pa)"})

rho = pd.DataFrame([cp.PropsSI('D', 'T', float(data['Temp. Aire (°C)'].iloc[i]) + T_0,
                               'P', float(P["Presión Dif. (Pa)"].iloc[i]) + P_0, 'Air')
                               for i in range(data.shape[0])], columns=["Rho (kg/m3)"])
mu = pd.DataFrame([cp.PropsSI('V', 'T', float(data['Temp. Aire (°C)'].iloc[i]) + T_0,
                               'D', float(rho['Rho (kg/m3)'].iloc[i]), 'Air') 
                               for i in range(data.shape[0])], columns=["Mu (Pa*s)"])

data = data.join([P, rho, mu]).drop(columns=["Presión Dif. (mbar)"])
data

,Scan Sweep Time (s),Temp. Entrada (°C),Vel. Aire (m/s),Temp. Aire (°C),Temp. Salida (°C),Presión Dif. (Pa),Rho (kg/m3),Mu (Pa*s)
0,2024-09-25 09:49:13.522,14.079405,6.472987,13.120901,17.538146,99.246850,1.234817,0.000018
1,2024-09-25 09:49:23.515,14.078369,6.656212,13.121559,17.564700,116.513805,1.235024,0.000018
2,2024-09-25 09:49:33.515,14.084538,6.437116,13.121187,17.573132,121.386540,1.235085,0.000018
3,2024-09-25 09:49:43.515,14.115000,6.546101,13.121459,17.570347,105.121852,1.234886,0.000018
4,2024-09-25 09:49:53.515,14.115085,6.437106,13.121394,17.589255,113.813643,1.234992,0.000018
...,...,...,...,...,...,...,...,...
2193,2024-09-25 15:54:43.516,30.401201,6.624077,18.110073,19.313422,-99.329177,1.211225,0.000018
2194,2024-09-25 15:54:53.516,30.379024,6.654350,17.866438,19.329506,-81.157420,1.212460,0.000018
2195,2024-09-25 15:55:03.516,30.373109,6.654364,18.111046,19.376573,-89.140490,1.211343,0.000018
2196,2024-09-25 15:55:13.516,30.396446,6.623994,18.111007,19.408551,-102.191115,1.211187,0.000018


Se sabe que la ecuación de Ergun es la siguiente:

$\frac{\Delta P}{L} = \frac{150 \mu (1-\epsilon)^2}{\phi^2 d^2_p \epsilon^3} v + \frac{1.75 \rho (1-\epsilon)}{\phi d_p \epsilon^3} v^2$

Despejando la presión diferencial se obtiene que:

$\Delta P = 150 \mu v \frac{L(1-\epsilon)^2}{\phi^2 d_p^2 \epsilon^3} + 1.75 \rho v^2 \frac{L(1-\epsilon)}{\phi d_p \epsilon^3}$

Luego, se optimiza para estimar los datos del lecho tal que minimice el error.

In [94]:
def ergun_loss(params, data):
    phi, dp, epsilon = params
    velocidad = np.array(data["Vel. Aire (m/s)"])
    delta_P_observada = np.array(data["Presión Dif. (Pa)"])
    rho = np.array(data["Rho (kg/m3)"])
    mu = np.array(data["Mu (Pa*s)"])
    L = 1.5
    
    # Pérdida de presión teórica según Ergun
    term_viscoso = (150 * mu * (1 - epsilon)**2) / (phi**2 * dp**2 * epsilon**3)
    term_inercial = (1.75 * rho * (1 - epsilon)) / (phi * dp * epsilon**3)
    delta_P_calculada = L * (term_viscoso * velocidad + term_inercial * velocidad**2)
    
    # Error entre valores calculados y observados
    error = np.sum((delta_P_calculada - delta_P_observada)**2)
    return error

params_iniciales = [0.8, 0.05, 0.4]
bounds = [(0.1, 1.0),   # phi
          (0.001, 0.1), # dp
          (0.5, 0.9)]  # epsilon
resultado = minimize(ergun_loss, params_iniciales, args=data, bounds=bounds)
print(resultado)


      fun: 102388733.19860311
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.57121544e+08, -1.57121566e+09, -2.09483081e+09])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 8
      nit: 1
     njev: 2
   status: 0
  success: True
        x: array([1. , 0.1, 0.9])


Teniendo los datos estimados se compara con la presión experimental el cálculo de la ecuación de Ergun

In [114]:
phi, dp, epsilon = resultado["x"]
L = 1.5
    
# Pérdida de presión teórica según Ergun
term_viscoso = (150 * data["Mu (Pa*s)"] * (1 - epsilon)**2) / (phi**2 * dp**2 * epsilon**3)
term_inercial = (1.75 * data["Rho (kg/m3)"] * (1 - epsilon)) / (phi * dp * epsilon**3)
delta_P_calculada = L * (term_viscoso * data["Vel. Aire (m/s)"] + term_inercial * data["Vel. Aire (m/s)"]**2)
# delta_P_calculada = pd.DataFrame(delta_P_calculada).rename(["Presión Teórica (Pa)"])
data = data.join(pd.DataFrame(delta_P_calculada, columns=["Presión Teórica (Pa)"]))
data = data.join(pd.DataFrame(abs(data["Presión Dif. (Pa)"]-data["Presión Teórica (Pa)"])/data["Presión Dif. (Pa)"], columns=["Error Relativo"]))
data

,Scan Sweep Time (s),Temp. Entrada (°C),Vel. Aire (m/s),Temp. Aire (°C),Temp. Salida (°C),Presión Dif. (Pa),Rho (kg/m3),Mu (Pa*s),Presión Teórica (Pa),Error Relativo
0,2024-09-25 09:49:13.522,14.079405,6.472987,13.120901,17.538146,99.246850,1.234817,0.000018,186.336114,0.877502
1,2024-09-25 09:49:23.515,14.078369,6.656212,13.121559,17.564700,116.513805,1.235024,0.000018,197.066389,0.691357
2,2024-09-25 09:49:33.515,14.084538,6.437116,13.121187,17.573132,121.386540,1.235085,0.000018,184.316885,0.518429
3,2024-09-25 09:49:43.515,14.115000,6.546101,13.121459,17.570347,105.121852,1.234886,0.000018,190.579564,0.812940
4,2024-09-25 09:49:53.515,14.115085,6.437106,13.121394,17.589255,113.813643,1.234992,0.000018,184.302411,0.619335
...,...,...,...,...,...,...,...,...,...,...
2193,2024-09-25 15:54:43.516,30.401201,6.624077,18.110073,19.313422,-99.329177,1.211225,0.000018,191.408603,-2.927013
2194,2024-09-25 15:54:53.516,30.379024,6.654350,17.866438,19.329506,-81.157420,1.212460,0.000018,193.358759,-3.382515
2195,2024-09-25 15:55:03.516,30.373109,6.654364,18.111046,19.376573,-89.140490,1.211343,0.000018,193.181540,-3.167158
2196,2024-09-25 15:55:13.516,30.396446,6.623994,18.111007,19.408551,-102.191115,1.211187,0.000018,191.397772,-2.872939


In [116]:
np.sum(data["Error Relativo"])/data.shape[0]

-0.8925577726707616

Se nota que existe una muy gran diferencia entre los valores teóricos y los experimentales, por lo que la ecaución no es una buena aproximación.

In [115]:
data.corr()["Presión Dif. (Pa)"]

Temp. Entrada (°C)      0.665450
Vel. Aire (m/s)         0.820089
Temp. Aire (°C)         0.963905
Temp. Salida (°C)       0.736789
Presión Dif. (Pa)       1.000000
Rho (kg/m3)            -0.963999
Mu (Pa*s)               0.964242
Presión Teórica (Pa)    0.077703
Error Relativo          0.951328
Name: Presión Dif. (Pa), dtype: float64

Observando las correlaciones, se nota que el factor que tiene la mayor incidencia en el resultado en la presión es la temperatura del aire, y las propiedades asociadas (densidad y viscosidad).

El factor menos incidente es la temperatura de entrada.

# Pregunta 2
Liquid  carbon  dioxide  enters  the  tubes  of  a  shell  and  tube  heat  exchanger  at  a  temperature of 0°C and a flow rate of 30 kg/s. The carbon dioxide is heated with a flow of city water that enters the shell side of the heat exchanger at a temperature of 40°C and a flow rate of 31 kg/s. The heat exchanger is a single shell, two-tube pass configuration with a 25in shell. The tubes are ¾in 10BWG laid out on a 1in triangular pitch. The heat exchanger is 2m long and contains 2 baffles per meter of length.
* Asumiendo presión atmosférica en la entrada

In [34]:
T_CO2 = (0*u.degC).to(u.K)
dm_CO2 = 30*u.kg/u.s
T_CityWater = (40*u.degC).to(u.K)
dm_CityWater = 31*u.kg/u.s

d_shell = (25*u.inch).to(u.m)
do_tube = (0.75*u.inch).to(u.m)
t_tube = t_from_gauge(10, schedule='BWG')
di_tube = do_tube - t_tube
l = 2*u.m
A_shell = np.pi * d_shell * l
A_tube = np.pi * di_tube * l

# Fouling Factor
R_CO2 = 0.000176*u.m**2*u.K/u.W
R_CityWater = 0.000352*u.m**2*u.K/u.W


In [43]:
P_CO2 = cp.PropsSI('P', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.Pa

cp_CO2 = cp.PropsSI('Cpmass', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.J/u.kg/u.K
cp_CityWater = cp.PropsSI('Cpmass', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.J/u.kg/u.K
Cmin = min(dm_CO2*cp_CO2, dm_CityWater*cp_CityWater)
Cmax = max(dm_CO2*cp_CO2, dm_CityWater*cp_CityWater)
Cr = Cmin/Cmax

rho_CO2 = cp.PropsSI('D', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.kg/u.m**3
mu_CO2 = cp.PropsSI('V', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.Pa*u.s

rho_CityWater = cp.PropsSI('D', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.kg/u.m**3
mu_CityWater = cp.PropsSI('V', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.Pa*u.s
k_CityWater = cp.PropsSI('L', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.W/u.m/u.K

V_CO2 = dm_CO2/(rho_CO2*A_tube)
V_CityWater = dm_CityWater/(rho_CityWater*A_shell)

Re_CityWater = fld.core.Reynolds(V_CityWater, d_shell, rho_CityWater, mu_CityWater).to(u.dimensionless)
Pr_CityWater = fld.core.Prandtl(Cp=cp_CityWater, mu=mu_CityWater, k=k_CityWater).to(u.dimensionless)
print(Re_CityWater, Pr_CityWater)

7558.734639617106 dimensionless 4.340630370366401 dimensionless


In [ ]:
Qmax = Cmin*(T_CityWater-T_CO2)

# Single shell double tube heat exchanger
e = lambda NTU, Cr: 2*(1+Cr+np.sqrt(1+Cr**2)*(1+np.exp(-NTU*np.sqrt(1+Cr**2)))/(1-np.exp(-NTU*np.sqrt(1+Cr**2))))**-1


a. The outlet temperatures and pressure drops of each fluid in the heat exchanger for the case of a brand new heat exchanger just put into service.

In [ ]:
print(Re_CO2.to(u.dimensionless))
print

47555.105742957145 dimensionless


b. The outlet temperatures of each fluid after the heat exchanger has been in continuous service for one year without maintenancece.

c. If the outlet temperature of the carbon dioxide must be 20°C or higher, is this heat exchanger sufficient for the required duty?

# Pregunta 3
Considere el plano hidráulico de la localidad de Hualañé y la distribución de diámetros determinada en la Tarea 2. Seleccione una bomba de uno de los dos catalogos disponibles, que permita entregar el caudal y nivel de presión requerido (6pts). Identifique el punto de operación de la bomba (6pts)

# Bibliografía